In [44]:
import requests
import lxml.html as lx
import pandas as pd

In [45]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
docs = lx.fromstring(page.content)
tr_elements = docs.xpath('//tr')

In [46]:
len(tr_elements[0]), len(tr_elements[1]), len(tr_elements[5])

(3, 3, 3)

In [47]:
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


In [48]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [49]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

Postal code\n           Borough\n                Neighborhood\n
0         M1A\n      Not assigned\n                            \n
1         M2A\n      Not assigned\n                            \n
2         M3A\n        North York\n                   Parkwoods\n
3         M4A\n        North York\n            Victoria Village\n
4         M5A\n  Downtown Toronto\n  Regent Park / Harbourfront\n

In [50]:
df = df.replace('\n',' ', regex=True)
df.head()

Postal code\n          Borough\n               Neighborhood\n
0          M1A       Not assigned                              
1          M2A       Not assigned                              
2          M3A         North York                    Parkwoods 
3          M4A         North York             Victoria Village 
4          M5A   Downtown Toronto   Regent Park / Harbourfront

In [51]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [53]:
df['Borough'][0]

'Not assigned '

In [54]:
df.drop(df.index[df['Borough'] == 'Not assigned '], inplace = True)
df.drop(df.index[df['PostalCode'] == ' '], inplace= True)
df = df.reset_index(drop=True)

df.head(10)

PostalCode            Borough                                   Neighborhood
0       M3A         North York                                      Parkwoods 
1       M4A         North York                               Victoria Village 
2       M5A   Downtown Toronto                     Regent Park / Harbourfront 
3       M6A         North York              Lawrence Manor / Lawrence Heights 
4       M7A   Downtown Toronto   Queen's Park / Ontario Provincial Government 
5       M9A          Etobicoke                               Islington Avenue 
6       M1B        Scarborough                                Malvern / Rouge 
7       M3B         North York                                      Don Mills 
8       M4B          East York               Parkview Hill / Woodbine Gardens 
9       M5B   Downtown Toronto                       Garden District, Ryerson

In [55]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).reset_index()
df.columns = ['PostalCode','Borough','Neighborhood']
df.head(10)

PostalCode       Borough                                       Neighborhood
0       M1B   Scarborough                                    Malvern / Rouge 
1       M1C   Scarborough           Rouge Hill / Port Union / Highland Creek 
2       M1E   Scarborough                Guildwood / Morningside / West Hill 
3       M1G   Scarborough                                             Woburn 
4       M1H   Scarborough                                          Cedarbrae 
5       M1J   Scarborough                                Scarborough Village 
6       M1K   Scarborough      Kennedy Park / Ionview / East Birchmount Park 
7       M1L   Scarborough                  Golden Mile / Clairlea / Oakridge 
8       M1M   Scarborough   Cliffside / Cliffcrest / Scarborough Village W...
9       M1N   Scarborough                       Birch Cliff / Cliffside West

In [56]:
df.loc[df['Neighborhood'] == 'Not assigned ', 'Neighborhood'] = df['Borough']

In [57]:
df.head()

PostalCode       Borough                               Neighborhood
0       M1B   Scarborough                            Malvern / Rouge 
1       M1C   Scarborough   Rouge Hill / Port Union / Highland Creek 
2       M1E   Scarborough        Guildwood / Morningside / West Hill 
3       M1G   Scarborough                                     Woburn 
4       M1H   Scarborough                                  Cedarbrae

In [59]:
df.shape

(103, 3)

In [64]:
df['Neighborhood'][0].split('/')

['Malvern ', ' Rouge ']

In [69]:
df = df.replace('/',',', regex=True)

In [70]:
df.head()

PostalCode       Borough                               Neighborhood
0       M1B   Scarborough                            Malvern , Rouge 
1       M1C   Scarborough   Rouge Hill , Port Union , Highland Creek 
2       M1E   Scarborough        Guildwood , Morningside , West Hill 
3       M1G   Scarborough                                     Woburn 
4       M1H   Scarborough                                  Cedarbrae

In [71]:
df.shape

(103, 3)

In [72]:
df.to_csv(r'df_canada.csv')